In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import category_encoders as ce

data = pd.read_csv('C:/Users/Lucas/Documents/Programacion/NoCountry/dbcompleta.csv')

# map outcome_types  0 or 1
def map_adopted(outcome_type):
    if outcome_type in ['Adoption', 'Return to Owner', 'Rto-Adopt']:
        return 1
    else:
        return 0

# Create the 'adopted' column based on the conditions
data['adopted'] = data['outcome_type'].apply(lambda x: map_adopted(x))

print (f"****Columnas despues de la limpieza y trasformacion de datos****")
print(data.head())



****Columnas despues de la limpieza y trasformacion de datos****
  animal_id         datetime_obj   
0   A786884  2019-01-03 16:19:00  \
1   A706918  2015-07-05 12:59:00   
2   A682524  2014-06-29 10:38:00   
3   A823407  2020-09-23 10:49:00   
4   A823407  2020-09-23 10:49:00   

                                      found_location intake_type   
0                2501 Magin Meadow Dr in Austin (TX)       Stray  \
1                   9409 Bluegrass Dr in Austin (TX)       Stray   
2                      800 Grove Blvd in Austin (TX)       Stray   
3  Rosewood Avenue And Poquito Street in Austin (TX)       Stray   
4  Rosewood Avenue And Poquito Street in Austin (TX)       Stray   

  intake_condition animal_type sex_upon_intake age_upon_intake   
0           Normal         Dog   Neutered Male         2 years  \
1           Normal         Dog   Spayed Female         8 years   
2           Normal         Dog   Neutered Male         4 years   
3           Normal         Cat   Intact Femal

In [2]:


# Elimina las columnas especificadas
columns_to_drop = ['animal_id', 'animal_id','age_upon_intake', 'age_upon_outcome', 'animal_type.1','found_location','animal_id.1','color.1', 'breed.1', 'datetime_obj', 'datetime_obj.1', 'outcome_type']
data.drop(columns=columns_to_drop, inplace=True)



In [3]:
# Separa la columna sex_upon_intake en sex y reprod.
data[['sex_intake', 'reprod_intake']] = data['sex_upon_intake'].str.split(' ', expand=True)

# Separa la columna sex_upon_outcome en sex y reprod.
data[['sex_outcome', 'reprod_outcome']] = data['sex_upon_outcome'].str.split(' ', expand=True)


# Elimina las columnas sobrantes
data.drop(columns=['sex_upon_intake', 'sex_upon_outcome', 'sex_outcome', 'sex_intake' ], inplace=True)




In [4]:
# Identificar las filas donde reprod_intake tiene un valor y reprod_outcome está vacío
mask_intake_empty_outcome = data['reprod_intake'].notnull() & data['reprod_outcome'].isnull()

# Llenar reprod_outcome con el valor de reprod_intake donde corresponda
data.loc[mask_intake_empty_outcome, 'reprod_outcome'] = data.loc[mask_intake_empty_outcome, 'reprod_intake']

# Identificar las filas donde reprod_outcome tiene un valor y reprod_intake está vacío
mask_outcome_empty_intake = data['reprod_outcome'].notnull() & data['reprod_intake'].isnull()

# Llenar reprod_intake con el valor de reprod_outcome donde corresponda
data.loc[mask_outcome_empty_intake, 'reprod_intake'] = data.loc[mask_outcome_empty_intake, 'reprod_outcome']


In [5]:

# Contar los valores en la columna 'sex_intake'
sex_intake_counts = data['reprod_intake'].value_counts()

# Contar los valores en la columna 'sex_outcome'
sex_outcome_counts = data['reprod_outcome'].value_counts()

# Mostrar los resultados
print("Conteo de valores en la columna 'sex_intake':")
print(sex_intake_counts)

print("\nConteo de valores en la columna 'sex_outcome':")
print(sex_outcome_counts)


Conteo de valores en la columna 'sex_intake':
reprod_intake
Male      103251
Female     89952
Name: count, dtype: int64

Conteo de valores en la columna 'sex_outcome':
reprod_outcome
Male      103251
Female     89952
Name: count, dtype: int64


In [6]:
data.drop(columns=['reprod_intake' ], inplace=True)

In [7]:
import numpy as np

# Contar el número de valores Male y Female en la columna reprod_outcome
male_count = data['reprod_outcome'].value_counts().get('Male', 0)
female_count = data['reprod_outcome'].value_counts().get('Female', 0)

# Calcular la cantidad de valores a llenar con Male y Female para mantener una proporción del 50%
total_null_values = data['reprod_outcome'].isnull().sum()
male_to_fill = int(total_null_values / 2)
female_to_fill = total_null_values - male_to_fill

# Crear un array con valores Male y Female de acuerdo a la proporción
values_to_fill = np.array(['Male'] * male_to_fill + ['Female'] * female_to_fill)

# Llenar valores vacíos de forma aleatoria con Male o Female
np.random.shuffle(values_to_fill)
data.loc[data['reprod_outcome'].isnull(), 'reprod_outcome'] = values_to_fill


In [8]:
# Contar los valores en la columna 'sex_outcome'
sex_outcome_counts = data['reprod_outcome'].value_counts()
print("\nConteo de valores en la columna 'sex_outcome':")
print(sex_outcome_counts)


Conteo de valores en la columna 'sex_outcome':
reprod_outcome
Male      109468
Female     96169
Name: count, dtype: int64


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205637 entries, 0 to 205636
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   intake_type       204822 non-null  object
 1   intake_condition  204822 non-null  object
 2   animal_type       204822 non-null  object
 3   breed             204822 non-null  object
 4   color             204822 non-null  object
 5   adopted           205637 non-null  int64 
 6   reprod_outcome    205637 non-null  object
dtypes: int64(1), object(6)
memory usage: 11.0+ MB


In [10]:
# Eliminar las filas con valores nulos en cualquier columna
data = data.dropna()



In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 204822 entries, 0 to 205415
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   intake_type       204822 non-null  object
 1   intake_condition  204822 non-null  object
 2   animal_type       204822 non-null  object
 3   breed             204822 non-null  object
 4   color             204822 non-null  object
 5   adopted           204822 non-null  int64 
 6   reprod_outcome    204822 non-null  object
dtypes: int64(1), object(6)
memory usage: 12.5+ MB


In [12]:
# Restablecer los índices en data_encoded
data.reset_index(drop=True, inplace=True)

In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204822 entries, 0 to 204821
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   intake_type       204822 non-null  object
 1   intake_condition  204822 non-null  object
 2   animal_type       204822 non-null  object
 3   breed             204822 non-null  object
 4   color             204822 non-null  object
 5   adopted           204822 non-null  int64 
 6   reprod_outcome    204822 non-null  object
dtypes: int64(1), object(6)
memory usage: 10.9+ MB


In [14]:
# Selección de las columnas categóricas a codificar
categorical_cols = ['breed', 'color', 'animal_type', 'intake_condition', 'intake_type', 'reprod_outcome']

# Crear un codificador BinaryEncoder
encoder = ce.BinaryEncoder(cols=categorical_cols)

# Aplicar el codificador y guardar los datos transformados en un nuevo DataFrame
data_encoded = encoder.fit_transform(data)

In [15]:
data_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204822 entries, 0 to 204821
Data columns (total 36 columns):
 #   Column              Non-Null Count   Dtype
---  ------              --------------   -----
 0   intake_type_0       204822 non-null  int64
 1   intake_type_1       204822 non-null  int64
 2   intake_type_2       204822 non-null  int64
 3   intake_condition_0  204822 non-null  int64
 4   intake_condition_1  204822 non-null  int64
 5   intake_condition_2  204822 non-null  int64
 6   intake_condition_3  204822 non-null  int64
 7   intake_condition_4  204822 non-null  int64
 8   animal_type_0       204822 non-null  int64
 9   animal_type_1       204822 non-null  int64
 10  animal_type_2       204822 non-null  int64
 11  breed_0             204822 non-null  int64
 12  breed_1             204822 non-null  int64
 13  breed_2             204822 non-null  int64
 14  breed_3             204822 non-null  int64
 15  breed_4             204822 non-null  int64
 16  breed_5             

In [16]:
# Guarda el DataFrame resultante en un nuevo archivo CSV
data_encoded.to_csv('data.csv', index=False)